In [2]:
import os
import json
from typing import Optional, List, Dict, Any
from dataclasses import dataclass
from enum import Enum
import asyncio

In [3]:
# ============================================
# Chat LLM Orchestrator
# ============================================

class LLMProvider(Enum):
    """사용 가능한 LLM 제공자"""
    OPENAI = "openai"
    ANTHROPIC = "anthropic"
    GOOGLE = "google"
    COHERE = "cohere"

@dataclass
class LLMConfig:
    """LLM 설정"""
    provider: LLMProvider
    model: str
    api_key: str
    temperature: float = 0.7
    max_tokens: int = 2048
    timeout: int = 30

@dataclass
class Message:
    """채팅 메시지"""
    role: str  # "user", "assistant", "system"
    content: str
    timestamp: Optional[float] = None

@dataclass
class ChatResponse:
    """LLM 응답"""
    content: str
    provider: LLMProvider
    model: str
    usage: Dict[str, int]  # token usage
    
class ChatLLMOrchestrator:
    """여러 LLM을 조율하는 오케스트레이터"""
    
    def __init__(self, configs: List[LLMConfig], primary_provider: LLMProvider = None):
        """
        Args:
            configs: LLM 설정 리스트
            primary_provider: 주요 LLM 제공자
        """
        self.llm_configs: Dict[LLMProvider, LLMConfig] = {
            config.provider: config for config in configs
        }
        self.primary_provider = primary_provider or configs[0].provider
        self.conversation_history: List[Message] = []
        self.response_cache: Dict[str, ChatResponse] = {}
        
    def add_message(self, role: str, content: str) -> None:
        """대화 히스토리에 메시지 추가"""
        import time
        message = Message(role=role, content=content, timestamp=time.time())
        self.conversation_history.append(message)
    
    def get_conversation_history(self) -> List[Dict[str, str]]:
        """대화 히스토리 반환"""
        return [
            {"role": msg.role, "content": msg.content}
            for msg in self.conversation_history
        ]
    
    async def query_llm(
        self,
        provider: LLMProvider,
        system_prompt: Optional[str] = None,
        use_cache: bool = True
    ) -> ChatResponse:
        """
        특정 LLM에 쿼리 전송
        
        Args:
            provider: 사용할 LLM 제공자
            system_prompt: 시스템 프롬프트
            use_cache: 캐시 사용 여부
            
        Returns:
            ChatResponse: LLM 응답
        """
        if provider not in self.llm_configs:
            raise ValueError(f"Unknown provider: {provider}")
        
        # 캐시 확인
        cache_key = self._generate_cache_key(provider, system_prompt)
        if use_cache and cache_key in self.response_cache:
            return self.response_cache[cache_key]
        
        config = self.llm_configs[provider]
        
        # 메시지 준비
        messages = []
        if system_prompt:
            messages.append({"role": "system", "content": system_prompt})
        messages.extend(self.get_conversation_history())
        
        # 비동기로 LLM 호출 시뮬레이션
        response = await self._call_llm_api(provider, config, messages)
        
        # 캐시에 저장
        if use_cache:
            self.response_cache[cache_key] = response
        
        return response
    
    async def query_primary(
        self,
        system_prompt: Optional[str] = None
    ) -> ChatResponse:
        """주요 LLM에 쿼리"""
        return await self.query_llm(self.primary_provider, system_prompt)
    
    async def query_all(
        self,
        system_prompt: Optional[str] = None
    ) -> Dict[LLMProvider, ChatResponse]:
        """모든 LLM에 쿼리"""
        tasks = [
            self.query_llm(provider, system_prompt)
            for provider in self.llm_configs.keys()
        ]
        results = await asyncio.gather(*tasks)
        return {
            provider: result
            for provider, result in zip(self.llm_configs.keys(), results)
        }
    
    async def compare_responses(
        self,
        system_prompt: Optional[str] = None
    ) -> Dict[str, Any]:
        """모든 LLM의 응답 비교"""
        responses = await self.query_all(system_prompt)
        
        return {
            "responses": {
                provider.value: response.content
                for provider, response in responses.items()
            },
            "usage": {
                provider.value: response.usage
                for provider, response in responses.items()
            },
            "comparison": {
                "total_providers": len(responses),
                "primary_response": responses[self.primary_provider].content
            }
        }
    
    def clear_history(self) -> None:
        """대화 히스토리 초기화"""
        self.conversation_history.clear()
    
    def clear_cache(self) -> None:
        """캐시 초기화"""
        self.response_cache.clear()
    
    def _generate_cache_key(self, provider: LLMProvider, system_prompt: Optional[str]) -> str:
        """캐시 키 생성"""
        history_str = json.dumps(self.get_conversation_history())
        prompt_str = system_prompt or ""
        key_data = f"{provider.value}_{prompt_str}_{history_str}"
        import hashlib
        return hashlib.md5(key_data.encode()).hexdigest()
    
    async def _call_llm_api(
        self,
        provider: LLMProvider,
        config: LLMConfig,
        messages: List[Dict[str, str]]
    ) -> ChatResponse:
        """
        LLM API 호출 (구현 예시)
        
        실제 구현에서는 각 제공자의 SDK 사용
        """
        # 시뮬레이션: 실제로는 API 호출
        await asyncio.sleep(0.5)  # API 호출 시뮬레이션
        
        response_content = f"Response from {provider.value} ({config.model})"
        
        return ChatResponse(
            content=response_content,
            provider=provider,
            model=config.model,
            usage={
                "prompt_tokens": 10,
                "completion_tokens": 20,
                "total_tokens": 30
            }
        )

# ============================================
# 사용 예시
# ============================================

async def main():
    """Chat LLM Orchestrator 사용 예시"""
    
    # LLM 설정
    configs = [
        LLMConfig(
            provider=LLMProvider.OPENAI,
            model="gpt-4",
            api_key=os.getenv("OPENAI_API_KEY", "dummy-key"),
            temperature=0.7
        ),
        LLMConfig(
            provider=LLMProvider.ANTHROPIC,
            model="claude-3",
            api_key=os.getenv("ANTHROPIC_API_KEY", "dummy-key"),
            temperature=0.5
        ),
        LLMConfig(
            provider=LLMProvider.GOOGLE,
            model="gemini-pro",
            api_key=os.getenv("GOOGLE_API_KEY", "dummy-key"),
            temperature=0.8
        ),
    ]
    
    # 오케스트레이터 생성
    orchestrator = ChatLLMOrchestrator(
        configs=configs,
        primary_provider=LLMProvider.OPENAI
    )
    
    # 대화 추가
    orchestrator.add_message("user", "안녕하세요, LLM 오케스트레이터입니다.")
    
    print("=" * 50)
    print("Chat LLM Orchestrator 시작")
    print("=" * 50)
    
    # 1. 주요 LLM에만 쿼리
    print("\n[1] 주요 LLM(OpenAI) 응답:")
    response = await orchestrator.query_primary(
        system_prompt="당신은 유용한 어시스턴트입니다."
    )
    print(f"Provider: {response.provider.value}")
    print(f"Content: {response.content}")
    print(f"Usage: {response.usage}")
    
    # 2. 모든 LLM 비교
    print("\n[2] 모든 LLM 응답 비교:")
    comparison = await orchestrator.compare_responses(
        system_prompt="당신은 유용한 어시스턴트입니다."
    )
    print(json.dumps(comparison, indent=2, ensure_ascii=False))
    
    # 3. 대화 히스토리 확인
    print("\n[3] 대화 히스토리:")
    for i, msg in enumerate(orchestrator.conversation_history, 1):
        print(f"  {i}. [{msg.role}] {msg.content}")
    
    # 4. 캐시 상태
    print(f"\n[4] 캐시 크기: {len(orchestrator.response_cache)} items")

# 실행
if __name__ == "__main__":
    asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop